In [1]:
import pickle
import pandas as pd

In [2]:
with open('./data/transformed_data/teams_stats.pickle', 'rb') as team_stats:
    teams_stats = pickle.load(team_stats)

In [3]:
with open('./data/transformed_data/players_stats.pickle', 'rb') as play_stats:
    players_stats = pickle.load(play_stats)

In [4]:
with open('./data/transformed_data/players_advanced_stats.pickle', 'rb') as play_advanced_stats:
    players_advanced_stats = pickle.load(play_advanced_stats)

In [5]:
game_types = ["playoffs", "leagues"]
#years = list(range(2001, 2023))
years = list(range(2001, 2002))
stats = {}
df_columns = ["ID", "age", "games_played_perc", "games_started_perc", "avg_minutes_played", "WS48", "team_successes", "defensive", "most_improved", "most_valuable", "sixth_man"]
df_template = pd.DataFrame(columns=df_columns)

In [6]:
for year in years:
    stats[year] = df_template

In [7]:
print(players_stats[2001]["leagues"].to_string())

                    Player  Age   Tm   G  GS    MP         ID
0       Mahmoud Abdul-Rauf   31  MEM  41   0  11.9  abdulma02
1        Tariq Abdul-Wahad   26  DEN  29  12  14.5  abdulta01
2      Shareef Abdur-Rahim   24  MEM  81  81  40.0  abdursh01
3           Cory Alexander   27  ORL  26   0   8.7  alexaco01
4       Courtney Alexander   23  WAS  65  24  21.3  alexaco02
7               Ray Allen*   25  MIL  82  82  38.2  allenra02
8             Rafer Alston   24  MIL  37   2   7.8  alstora01
9             John Amaechi   30  ORL  82  36  20.9  amaecjo01
10          Derek Anderson   26  SAS  82  82  34.9  anderde01
11          Kenny Anderson   30  BOS  33  28  25.7  anderke01
12           Nick Anderson   33  SAC  21   0   8.0  anderni01
13        Shandon Anderson   27  HOU  82  82  29.2  andersh01
14            Greg Anthony   33  POR  58   0  14.8  anthogr01
15       Darrell Armstrong   32  ORL  75  75  36.9  armstda01
16           Chucky Atkins   26  DET  81  75  29.2  atkinch01
17      

In [8]:
print(players_stats[2001]["playoffs"].to_string())

                   Player  Age   Tm   G  GS    MP         ID
0              Ray Allen*   25  MIL  18  18  42.7  allenra02
1            Rafer Alston   24  MIL   5   0   1.6  alstora01
2            John Amaechi   30  ORL   4   0   7.3  amaecjo01
3          Derek Anderson   26  SAS   7   7  27.7  anderde01
4            Greg Anthony   33  POR   2   0   8.5  anthogr01
5       Darrell Armstrong   32  ORL   4   4  41.8  armstda01
6           Stacey Augmon   32  POR   2   0  14.0  augmost01
7           William Avery   21  MIN   2   0   6.0  averywi01
8               Jon Barry   31  SAC   7   0   7.9  barryjo01
9               Raja Bell   24  PHI  15   0   8.3   bellra01
10        Jonathan Bender   20  IND   1   0   4.0  bendejo01
11           David Benoit   32  UTA   4   0  15.3  benoida01
12            Travis Best   28  IND   4   4  40.8   besttr01
13       Chauncey Billups   24  MIN   3   0   8.7  billuch01
14           Calvin Booth   24  DAL  10   0  13.7  boothca01
15            Bruce Bowe

In [15]:
# % GAMES PLAYED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    stats[year]['ID'] = [player for player in player_list]
    stats[year] = stats[year].set_index("ID")
    for player_id in player_list:
        games_played = 0
        team_games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    player_team = players_stats[year][game_type].query(f"ID == '{player_id}'")["Tm"].iloc[0]
                except:
                    pass
                try:
                    games_played += players_stats[year][game_type].query(f"ID == '{player_id}'")["G"].iloc[0]   
                except:
                    pass
                try:
                    team_games_played += teams_stats[year][game_type].query(f"Team == '{player_team}'")["G"].iloc[0]
                except:    
                    pass

        perc_games_played = games_played / team_games_played
            
        stats[year].loc[[f'{player_id}'], ['games_played_perc']] = perc_games_played

In [21]:
# % GAMES STARTED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        games_started = 0
        team_games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    player_team = players_stats[year][game_type].query(f"ID == '{player_id}'")["Tm"].iloc[0]
                except:
                    pass
                try:
                    games_started += players_stats[year][game_type].query(f"ID == '{player_id}'")["GS"].iloc[0]
                except:
                    pass
                try:    
                    team_games_played += teams_stats[year][game_type].query(f"Team == '{player_team}'")["G"].iloc[0]
                except:
                    pass

        perc_games_started = games_started / team_games_played
            
        stats[year].loc[[f'{player_id}'], ['games_started_perc']] = perc_games_started

In [22]:
# AVERAGE MINUTES PLAYED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        avg_minutes_played = 0
        avg_minutes_played_x_games = 0
        total_games = 0
        minutes_played_per_game = 0
        games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    minutes_played_per_game = players_stats[year][game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
                    games_played = players_stats[year][game_type].query(f"ID == '{player_id}'")["G"].iloc[0]
                except:
                    minutes_played_per_game += 0
                    games_played += 0
                
            avg_minutes_played_x_games += minutes_played_per_game * games_played
            total_games += games_played
            
        if total_games != 0:
            avg_minutes_played = round(avg_minutes_played_x_games/total_games, 2)
        else:
            avg_minutes_played = 0
        
        stats[year].loc[[f'{player_id}'], ['avg_minutes_played']] = avg_minutes_played

In [23]:
# WIN SHARES PER 48 MINUTES FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        minutes_played = 0
        win_shares = 0
        win_share_48 = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    minutes_played = players_stats[year][game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
                    win_shares = players_advanced_stats[year][game_type].query(f"ID == '{player_id}'")["WS"].iloc[0]
                except:
                    minutes_played += 0
                    win_shares += 0
            
        if minutes_played != 0:
            win_share_48 = round((win_shares/minutes_played) * 48, 2)
        else:
            win_share_48 = 0
        
        stats[year].loc[[f'{player_id}'], ['WS48']] = win_share_48

In [26]:
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        try:
            stats[year].loc[[f'{player_id}'], ['age']] = players_stats[year]["leagues"].query(f"ID == '{player_id}'")["Age"].iloc[0]
        except:
            stats[year].loc[[f'{player_id}'], ['age']] = players_stats[year]["playoffs"].query(f"ID == '{player_id}'")["Age"].iloc[0]

In [27]:
print(stats[2001])

          age games_played_perc games_started_perc avg_minutes_played   WS48   
ID                                                                             
duncati01  24               1.0                1.0              38.95  16.37  \
stricma01  30          0.578947           0.021053               13.1    2.2   
wolkoru01  27          0.414634           0.012195                9.0  -0.53   
anderni01  33          0.256098                0.0                8.0    0.0   
stewami01  25          0.297872                0.0               4.51   1.02   
...        ..               ...                ...                ...    ...   
gillke01   32          0.360465           0.302326               28.8  -0.67   
hamilri01  22           0.95122           0.512195               32.3   3.57   
feickja01  26          0.073171                0.0               24.8   0.39   
smithja01  23          0.109756                0.0                7.3   1.32   
stojape01  23          0.922222         